In [1]:
import pandas as pd
import numpy as np
import warnings
import tqdm
warnings.filterwarnings("ignore")

In [2]:
TIME_CUT = "1973-01-01"
SPLIT_DATE_TRAIN = "2020-01-01"
MAG_TH = 5
GEO_SPLIT = 1

In [3]:
df = pd.read_csv("../data/usgs_data_small.csv")
df.dropna(inplace=True)
df

,time,longitude,latitude,depth,mag
0,1940-01-06 20:04:35.170000,25.814000,35.373000,15.00,5.84
2,1940-01-06 09:15:39.210000,151.498000,45.077000,25.00,6.07
3,1940-01-05 09:42:55.570000,-116.367333,33.173167,6.00,3.42
4,1940-01-05 07:20:50.460000,-119.442000,32.929333,6.00,3.97
5,1940-01-04 21:44:55.390000,37.926000,40.415000,15.00,5.51
...,...,...,...,...,...
4329143,2023-09-24 02:20:39.520000,-117.314167,34.071333,16.11,1.09
4329144,2023-09-24 02:11:22.160000,-155.259000,58.195167,2.55,0.14
4329145,2023-09-24 02:10:46.071000,-119.668800,40.223400,8.40,0.70
4329146,2023-09-24 02:06:35.040000,-123.223333,39.279500,4.43,1.91


In [4]:
df["time"] = df["time"].apply(lambda x: x[:10])
df = df[df["time"] >= TIME_CUT]
df["time"] = pd.to_datetime(df["time"])
df

,time,longitude,latitude,depth,mag
69377,1973-01-07,-155.432333,19.771500,22.834,2.34
69378,1973-01-06,167.487000,-15.496000,123.000,5.30
69379,1973-01-06,48.256000,33.098000,63.000,4.80
69380,1973-01-06,-157.021500,20.207667,4.900,3.23
69381,1973-01-06,166.384000,-14.665000,36.000,6.10
...,...,...,...,...,...
4329143,2023-09-24,-117.314167,34.071333,16.110,1.09
4329144,2023-09-24,-155.259000,58.195167,2.550,0.14
4329145,2023-09-24,-119.668800,40.223400,8.400,0.70
4329146,2023-09-24,-123.223333,39.279500,4.430,1.91


In [5]:
df["longitude_disc"] = (df["longitude"] // GEO_SPLIT * GEO_SPLIT).astype(int)
df["latitude_disc"] = (df["latitude"] // GEO_SPLIT * GEO_SPLIT).astype(int)
df["pos"] = df["latitude_disc"].astype(str) + "_" + df["longitude_disc"].astype(str)

In [7]:
df

,time,longitude,latitude,depth,mag,longitude_disc,latitude_disc,pos
69377,1973-01-07,-155.432333,19.771500,22.834,2.34,-156,19,19_-156
69378,1973-01-06,167.487000,-15.496000,123.000,5.30,167,-16,-16_167
69379,1973-01-06,48.256000,33.098000,63.000,4.80,48,33,33_48
69380,1973-01-06,-157.021500,20.207667,4.900,3.23,-158,20,20_-158
69381,1973-01-06,166.384000,-14.665000,36.000,6.10,166,-15,-15_166
...,...,...,...,...,...,...,...,...
4329143,2023-09-24,-117.314167,34.071333,16.110,1.09,-118,34,34_-118
4329144,2023-09-24,-155.259000,58.195167,2.550,0.14,-156,58,58_-156
4329145,2023-09-24,-119.668800,40.223400,8.400,0.70,-120,40,40_-120
4329146,2023-09-24,-123.223333,39.279500,4.430,1.91,-124,39,39_-124


In [8]:
# https://www.usgs.gov/faqs/how-can-earthquake-have-negative-magnitude
df["mag"].min()

-9.99

In [9]:
# https://www.usgs.gov/observatories/hvo/news/volcano-watch-why-do-some-earthquakes-have-negative-depths
df["depth"].min()

-10.0

In [10]:
df["longitude"].min(), df["longitude"].max()

(-179.9997, 180.0)

In [11]:
df["latitude"].min(), df["latitude"].max()

(-84.422, 87.386)

In [12]:
df_tp = pd.read_csv("../data/all.csv")
df_tp.head()

,plate,lat,lon
0,am,30.754,132.824
1,am,30.970,132.965
2,am,31.216,133.197
3,am,31.515,133.500
4,am,31.882,134.042


In [13]:
plate2id = {plate: i+1 for i, plate in enumerate(df_tp["plate"].unique())}
df_tp["plate"] = df_tp["plate"].map(plate2id)

In [14]:
pd.DataFrame(plate2id.items(), columns=["plate", "plate_id"]).to_csv("../data/plate2id.csv", index=False)

In [15]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

In [16]:
def find_min_dist(df_tp, x, y):
    df_tp["dist"] = haversine_distance(x, y, df_tp["lat"], df_tp["lon"])
    plate = df_tp.loc[df_tp["dist"].idxmin(), "plate"]
    return df_tp["dist"].min(), plate

In [17]:
def add_region_info(df, df_tp):
    region2plate = {}
    region2dist = {}
    df["lat_cent"] = df["latitude_disc"] + 0.5
    df["lon_cent"] = df["longitude_disc"] + 0.5
    for pos in tqdm.tqdm(df["pos"].unique()):
        x, y = pos.split("_")
        x, y = float(x), float(y)
        dist, plate = find_min_dist(df_tp, x + 0.5, y + 0.5)
        region2plate[pos] = plate
        region2dist[pos] = dist
    df["plate_region"] = df["pos"].map(region2plate)
    df["dist_region"] = df["pos"].map(region2dist)
    return df

In [18]:
def add_tectonic_info(df, df_tp):
    coordinates = list(zip(df['latitude'], df['longitude']))
    results = list(tqdm.tqdm(map(lambda x: find_min_dist(df_tp, x[0], x[1]), coordinates), total=len(coordinates)))
    df[["dist", "plate"]] = results
    return df

In [19]:
def add_features(df, df_tp, mag_th):
    df_final = None
    df["time"] = pd.to_datetime(df["time"], format="mixed")
    df = add_region_info(df, df_tp)
    df = add_tectonic_info(df, df_tp)
    for pos in tqdm.tqdm(df["pos"].unique()):
        dfs = []
        tmp = df[df["pos"] == pos]
        tmp.sort_values("time", inplace=True)
        for time in tmp["time"].unique():
            tmp_t0 = tmp[tmp["time"] == time]
            t1 = time + pd.DateOffset(months=1)
            tmp_t1 = tmp[(tmp["time"] > time) & (tmp["time"] <= t1)]
            if tmp_t1.empty:
                max_mag = -1e8
            else:
                max_mag = tmp_t1["mag"].max()
            tmp_t0["label"] = 0 if max_mag < mag_th else 1
            dfs.append(tmp_t0)
        df_tmp = pd.concat(dfs)
        df_final = pd.concat([df_final, df_tmp])
    return df_final

In [20]:
df_final = add_features(df, df_tp, MAG_TH)

100%|██████████| 13103/13103 [2:21:41<00:00,  1.54it/s]  


In [23]:
df_final.to_csv("../data/with_features_final.csv", index=False)